# Building a movie recommendation system for telugu movies 

importin all required libraries

In [1]:
import pandas as pd
import numpy as np 
import ast
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from sklearn.metrics.pairwise import cosine_similarity
import pickle

C:\Users\User\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Reading data frames containg features  

In [2]:
movie=pd.read_csv('TMDB telugu movie details.csv',index_col=0)
genres=pd.read_csv('Genres.csv',index_col=0)
cast=pd.read_csv('cast_details.csv',index_col=0)


In [3]:
movie.head()

,id,genre_ids,title,overview,poster_path
0,579974,"[28, 18]",RRR,A fictional history of two legendary revolutio...,/nEufeZlyAOLqO2brrs0yeF1lgXO.jpg
1,256040,"[28, 12, 18, 14]",Bāhubali: The Beginning,The young Shivudu is left as a foundling in a ...,/9BAjt8nSSms62uOVYn1t3C3dVto.jpg
2,690957,"[28, 18, 53]",Pushpa: The Rise - Part 1,Pushpa Raj is a coolie who rises in the world ...,/r1yAzVQNbCbPTbB3GZFour9Qr0t.jpg
3,350312,"[28, 12, 14]",Bāhubali 2: The Conclusion,"When Mahendra, the son of Bāhubali, learns abo...",/21sC2assImQIYCEDA84Qh9d1RsK.jpg
4,901277,"[80, 53]",CSI Sanatan,When a buisness tycoon Vikram Chakravarthi get...,/yscCuJsNYDh2vHc6VrASnZcbaDw.jpg


In [4]:
cast.head()

,adult,gender,id,known_for_department,name,original_name,popularity,profile_path,cast_id,character,credit_id,order
0,False,2.0,579974,Acting,N. T. Rama Rao Jr.,N. T. Rama Rao Jr.,13.410,/kMz1SZFt7ynfL8kxs7mZZHkSzkc.jpg,1.0,Komaram Bheem,5c57c795c3a368478ca254ea,0.0
1,False,2.0,579974,Acting,Ram Charan,Ram Charan,15.840,/mNQA0qYtLeemo7mljd9RlG9jUMR.jpg,2.0,Alluri Sitarama Raju,5c57c7a192514151a965696f,1.0
2,False,2.0,579974,Acting,Ajay Devgn,Ajay Devgn,14.079,/vnHQQFzTjJ0sv14DGMIoKa3qTxN.jpg,12.0,Venkata Rama Raju,5c8f45940e0a261304586785,2.0
3,False,1.0,579974,Acting,Alia Bhatt,Alia Bhatt,10.690,/RBnTJPegPFLBS4VPsNLbf6iAoD.jpg,11.0,Sita,5c8f454fc3a36861194ddbeb,3.0
4,False,1.0,579974,Acting,Shriya Saran,Shriya Saran,7.211,/ioOgKBSCJhOdKoZpg86ld7zANbf.jpg,38.0,Sarojini,61051102fdc14600731f51cb,4.0


checking for null values in movie dataframe

In [5]:
movie.isnull().sum()

id               0
genre_ids        0
title            0
overview       227
poster_path    557
dtype: int64

movie contains null values of 230 overview columnn and 568 poster_path column 

In [6]:
  genres

,id,name
0,28,Action
1,12,Adventure
2,16,Animation
3,35,Comedy
4,80,Crime
5,99,Documentary
6,18,Drama
7,10751,Family
8,14,Fantasy
9,36,History


extracting geners columnn details by using genre_ids and storing as genre column in movie dataframe

In [7]:
id_name_dict = dict(zip(genres['id'], genres['name']))
movie['genre'] = [ [id_name_dict[i] for i in ast.literal_eval(row)] for row in movie['genre_ids'] ]


In [8]:
movie

,id,genre_ids,title,overview,poster_path,genre
0,579974,"[28, 18]",RRR,A fictional history of two legendary revolutio...,/nEufeZlyAOLqO2brrs0yeF1lgXO.jpg,"[Action, Drama]"
1,256040,"[28, 12, 18, 14]",Bāhubali: The Beginning,The young Shivudu is left as a foundling in a ...,/9BAjt8nSSms62uOVYn1t3C3dVto.jpg,"[Action, Adventure, Drama, Fantasy]"
2,690957,"[28, 18, 53]",Pushpa: The Rise - Part 1,Pushpa Raj is a coolie who rises in the world ...,/r1yAzVQNbCbPTbB3GZFour9Qr0t.jpg,"[Action, Drama, Thriller]"
3,350312,"[28, 12, 14]",Bāhubali 2: The Conclusion,"When Mahendra, the son of Bāhubali, learns abo...",/21sC2assImQIYCEDA84Qh9d1RsK.jpg,"[Action, Adventure, Fantasy]"
4,901277,"[80, 53]",CSI Sanatan,When a buisness tycoon Vikram Chakravarthi get...,/yscCuJsNYDh2vHc6VrASnZcbaDw.jpg,"[Crime, Thriller]"
...,...,...,...,...,...,...
2415,274754,[35],Andagadu,Though he's successful in his career composing...,/iR3pxOIigseQRfqCAmr4yKcLu7f.jpg,[Comedy]
2416,272227,[],Love Birds,Drama / Foreign - A love story about Arun and ...,NaN,[]
2417,270720,[10749],Pyar Mein Padipoyane,Chandra alias Chinna and Yuktha are childhood ...,NaN,[Romance]
2418,267689,[53],Billa Ranga,Billa Ranga is action thriller based movie in ...,/4k7oR6S8lZMNYuqBNseGp4ZZfZ9.jpg,[Thriller]


takig top 3 actors only from cast dataframe 

In [9]:
actors=cast.groupby('id')['original_name'].apply(lambda x: x.head(3))

In [10]:
actors=actors.reset_index()

In [11]:
actors.duplicated().sum()

0

takig top 3 actors only from cast dataframe and converting actors into list

In [12]:
actors=cast.groupby('id')['original_name'].apply(lambda x: x.head(3).tolist())

In [13]:
actors=actors.reset_index()

In [14]:
movie=movie.merge(actors,on='id')

creating new column called actore for readability and understaing

In [15]:
movie['actors']= movie['original_name']

In [16]:
movie

,id,genre_ids,title,overview,poster_path,genre,original_name,actors
0,579974,"[28, 18]",RRR,A fictional history of two legendary revolutio...,/nEufeZlyAOLqO2brrs0yeF1lgXO.jpg,"[Action, Drama]","[N. T. Rama Rao Jr., Ram Charan, Ajay Devgn]","[N. T. Rama Rao Jr., Ram Charan, Ajay Devgn]"
1,256040,"[28, 12, 18, 14]",Bāhubali: The Beginning,The young Shivudu is left as a foundling in a ...,/9BAjt8nSSms62uOVYn1t3C3dVto.jpg,"[Action, Adventure, Drama, Fantasy]","[Prabhas, Rana Daggubati, Tamannaah Bhatia]","[Prabhas, Rana Daggubati, Tamannaah Bhatia]"
2,690957,"[28, 18, 53]",Pushpa: The Rise - Part 1,Pushpa Raj is a coolie who rises in the world ...,/r1yAzVQNbCbPTbB3GZFour9Qr0t.jpg,"[Action, Drama, Thriller]","[Allu Arjun, Fahadh Faasil, Rashmika Mandanna]","[Allu Arjun, Fahadh Faasil, Rashmika Mandanna]"
3,350312,"[28, 12, 14]",Bāhubali 2: The Conclusion,"When Mahendra, the son of Bāhubali, learns abo...",/21sC2assImQIYCEDA84Qh9d1RsK.jpg,"[Action, Adventure, Fantasy]","[Prabhas, Rana Daggubati, Sathyaraj]","[Prabhas, Rana Daggubati, Sathyaraj]"
4,901277,"[80, 53]",CSI Sanatan,When a buisness tycoon Vikram Chakravarthi get...,/yscCuJsNYDh2vHc6VrASnZcbaDw.jpg,"[Crime, Thriller]","[Aadi Saikumar, Misha Narang, Ali Reza]","[Aadi Saikumar, Misha Narang, Ali Reza]"
...,...,...,...,...,...,...,...,...
2299,278035,"[18, 10751]",Swati Kiranam,An egotistical music teacher envious of the ex...,/nMtsgVTfaCdVQZOXx1Yh8Himj5M.jpg,"[Drama, Family]","[Mammootty, Radhika Sarathkumar, Master Manjun...","[Mammootty, Radhika Sarathkumar, Master Manjun..."
2300,274754,[35],Andagadu,Though he's successful in his career composing...,/iR3pxOIigseQRfqCAmr4yKcLu7f.jpg,[Comedy],"[Rajendra Prasad, Damini, Chandramohan]","[Rajendra Prasad, Damini, Chandramohan]"
2301,270720,[10749],Pyar Mein Padipoyane,Chandra alias Chinna and Yuktha are childhood ...,NaN,[Romance],"[Aadi Saikumar, Shanvi Shrivastava]","[Aadi Saikumar, Shanvi Shrivastava]"
2302,267689,[53],Billa Ranga,Billa Ranga is action thriller based movie in ...,/4k7oR6S8lZMNYuqBNseGp4ZZfZ9.jpg,[Thriller],"[Chiranjeevi, Mohan Babu, Swapna]","[Chiranjeevi, Mohan Babu, Swapna]"


ectracting directos for each movie in cast dataframe by masking dataframe

In [17]:
directors=cast[cast['known_for_department']=='Directing']

In [18]:
directors.head(2)

,adult,gender,id,known_for_department,name,original_name,popularity,profile_path,cast_id,character,credit_id,order
22,False,2.0,579974,Directing,S. S. Rajamouli,S. S. Rajamouli,7.548,/g7jXGAVT8T5vq81fO5JWDleDc75.jpg,74.0,Self,625c2adf3dd12600681b0327,22.0
45,False,2.0,256040,Directing,S. S. Rajamouli,S. S. Rajamouli,7.548,/g7jXGAVT8T5vq81fO5JWDleDc75.jpg,15.0,Spirit Seller (uncredited),55a1a089c3a3681cec001ce6,14.0


In [19]:
directors.shape

(167, 12)

since director dataframe has only 167 rows.if i merge with movie dataframe its shape will reduce as of directors
so not merging ditrectors with movie 

In [20]:
#movie.merge(directors,on ='id')

In [21]:
#movie['director']=movie['original_name_y']

In [22]:
movie

,id,genre_ids,title,overview,poster_path,genre,original_name,actors
0,579974,"[28, 18]",RRR,A fictional history of two legendary revolutio...,/nEufeZlyAOLqO2brrs0yeF1lgXO.jpg,"[Action, Drama]","[N. T. Rama Rao Jr., Ram Charan, Ajay Devgn]","[N. T. Rama Rao Jr., Ram Charan, Ajay Devgn]"
1,256040,"[28, 12, 18, 14]",Bāhubali: The Beginning,The young Shivudu is left as a foundling in a ...,/9BAjt8nSSms62uOVYn1t3C3dVto.jpg,"[Action, Adventure, Drama, Fantasy]","[Prabhas, Rana Daggubati, Tamannaah Bhatia]","[Prabhas, Rana Daggubati, Tamannaah Bhatia]"
2,690957,"[28, 18, 53]",Pushpa: The Rise - Part 1,Pushpa Raj is a coolie who rises in the world ...,/r1yAzVQNbCbPTbB3GZFour9Qr0t.jpg,"[Action, Drama, Thriller]","[Allu Arjun, Fahadh Faasil, Rashmika Mandanna]","[Allu Arjun, Fahadh Faasil, Rashmika Mandanna]"
3,350312,"[28, 12, 14]",Bāhubali 2: The Conclusion,"When Mahendra, the son of Bāhubali, learns abo...",/21sC2assImQIYCEDA84Qh9d1RsK.jpg,"[Action, Adventure, Fantasy]","[Prabhas, Rana Daggubati, Sathyaraj]","[Prabhas, Rana Daggubati, Sathyaraj]"
4,901277,"[80, 53]",CSI Sanatan,When a buisness tycoon Vikram Chakravarthi get...,/yscCuJsNYDh2vHc6VrASnZcbaDw.jpg,"[Crime, Thriller]","[Aadi Saikumar, Misha Narang, Ali Reza]","[Aadi Saikumar, Misha Narang, Ali Reza]"
...,...,...,...,...,...,...,...,...
2299,278035,"[18, 10751]",Swati Kiranam,An egotistical music teacher envious of the ex...,/nMtsgVTfaCdVQZOXx1Yh8Himj5M.jpg,"[Drama, Family]","[Mammootty, Radhika Sarathkumar, Master Manjun...","[Mammootty, Radhika Sarathkumar, Master Manjun..."
2300,274754,[35],Andagadu,Though he's successful in his career composing...,/iR3pxOIigseQRfqCAmr4yKcLu7f.jpg,[Comedy],"[Rajendra Prasad, Damini, Chandramohan]","[Rajendra Prasad, Damini, Chandramohan]"
2301,270720,[10749],Pyar Mein Padipoyane,Chandra alias Chinna and Yuktha are childhood ...,NaN,[Romance],"[Aadi Saikumar, Shanvi Shrivastava]","[Aadi Saikumar, Shanvi Shrivastava]"
2302,267689,[53],Billa Ranga,Billa Ranga is action thriller based movie in ...,/4k7oR6S8lZMNYuqBNseGp4ZZfZ9.jpg,[Thriller],"[Chiranjeevi, Mohan Babu, Swapna]","[Chiranjeevi, Mohan Babu, Swapna]"


selecting required columns for web application buiding 

In [23]:
movies=pd.DataFrame(movie[['id', 'title','genre', 'overview',  'actors','poster_path']])

In [24]:
movies.head(2)

,id,title,genre,overview,actors,poster_path
0,579974,RRR,"[Action, Drama]",A fictional history of two legendary revolutio...,"[N. T. Rama Rao Jr., Ram Charan, Ajay Devgn]",/nEufeZlyAOLqO2brrs0yeF1lgXO.jpg
1,256040,Bāhubali: The Beginning,"[Action, Adventure, Drama, Fantasy]",The young Shivudu is left as a foundling in a ...,"[Prabhas, Rana Daggubati, Tamannaah Bhatia]",/9BAjt8nSSms62uOVYn1t3C3dVto.jpg


In [25]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2304 entries, 0 to 2303
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           2304 non-null   int64 
 1   title        2304 non-null   object
 2   genre        2304 non-null   object
 3   overview     2089 non-null   object
 4   actors       2304 non-null   object
 5   poster_path  1808 non-null   object
dtypes: int64(1), object(5)
memory usage: 126.0+ KB


In [26]:
movies.isnull().sum()


id               0
title            0
genre            0
overview       215
actors           0
poster_path    496
dtype: int64

In [27]:
movies[pd.isnull(movies['overview']) ]

,id,title,genre,overview,actors,poster_path
20,1114702,Vimanam,"[Drama, Family]",NaN,"[Samuthirakani, Anasuya Bharadwaj, Meera Jasmine]",/d1FQfGMDGRiPqqG5MTC7DZi4joY.jpg
47,1032293,Anni Manchi Sakunamule,"[Comedy, Family, Romance]",NaN,"[Santosh Shoban, Malavika Nair, Vennela Kishore]",/rUX0hS51w2lIiMBVvfMHpJfKW6W.jpg
96,1090486,BRO,"[Comedy, Drama, Fantasy]",NaN,"[Pawan Kalyan, Sai Dharam Tej, Ketika Sharma]",/AhKWtkIjtPTaRawVaR1roNj1HxR.jpg
132,864221,Bhola Shankar,"[Drama, Action]",NaN,"[Chiranjeevi, Keerthy Suresh, Tamannaah Bhatia]",/kG63t05rRkIIRB0F2DNCExh3GwK.jpg
201,1034164,Nenu Student Sir!,"[Comedy, Romance, Thriller]",NaN,"[Bellamkonda Ganesh, Avantika Dassani, Samuthi...",/uCioTenCVELAe8csXyP0XcexQ7Q.jpg
...,...,...,...,...,...,...
2278,279497,Aggiramudu,[Romance],NaN,"[Venkatesh, Amala Akkineni, Gautami Tadimalla]",NaN
2281,279363,Mogudu Kaavali,[Family],NaN,"[Chiranjeevi, Gayathri]",NaN
2295,279287,Chakravarthy,[Drama],NaN,"[Chiranjeevi, Mohan Babu, Bhanupriya]",NaN
2297,279108,Hands Up,[Comedy],NaN,"[Nagendra Babu, Jayasudha, Chiranjeevi]",NaN


since dataframe having missing values droping missing values in dataframe

In [28]:
movies.dropna(inplace=True)

In [29]:
movies.shape

(1682, 6)

In [30]:
movies

,id,title,genre,overview,actors,poster_path
0,579974,RRR,"[Action, Drama]",A fictional history of two legendary revolutio...,"[N. T. Rama Rao Jr., Ram Charan, Ajay Devgn]",/nEufeZlyAOLqO2brrs0yeF1lgXO.jpg
1,256040,Bāhubali: The Beginning,"[Action, Adventure, Drama, Fantasy]",The young Shivudu is left as a foundling in a ...,"[Prabhas, Rana Daggubati, Tamannaah Bhatia]",/9BAjt8nSSms62uOVYn1t3C3dVto.jpg
2,690957,Pushpa: The Rise - Part 1,"[Action, Drama, Thriller]",Pushpa Raj is a coolie who rises in the world ...,"[Allu Arjun, Fahadh Faasil, Rashmika Mandanna]",/r1yAzVQNbCbPTbB3GZFour9Qr0t.jpg
3,350312,Bāhubali 2: The Conclusion,"[Action, Adventure, Fantasy]","When Mahendra, the son of Bāhubali, learns abo...","[Prabhas, Rana Daggubati, Sathyaraj]",/21sC2assImQIYCEDA84Qh9d1RsK.jpg
4,901277,CSI Sanatan,"[Crime, Thriller]",When a buisness tycoon Vikram Chakravarthi get...,"[Aadi Saikumar, Misha Narang, Ali Reza]",/yscCuJsNYDh2vHc6VrASnZcbaDw.jpg
...,...,...,...,...,...,...
2296,279110,Master,"[Action, Drama]",A popular college professor is unable to find ...,"[Chiranjeevi, Sakshi Shivananda, Puneet Issar]",/dZwuPQ3xU6jVpM4V2x8f2VeZYxt.jpg
2299,278035,Swati Kiranam,"[Drama, Family]",An egotistical music teacher envious of the ex...,"[Mammootty, Radhika Sarathkumar, Master Manjun...",/nMtsgVTfaCdVQZOXx1Yh8Himj5M.jpg
2300,274754,Andagadu,[Comedy],Though he's successful in his career composing...,"[Rajendra Prasad, Damini, Chandramohan]",/iR3pxOIigseQRfqCAmr4yKcLu7f.jpg
2302,267689,Billa Ranga,[Thriller],Billa Ranga is action thriller based movie in ...,"[Chiranjeevi, Mohan Babu, Swapna]",/4k7oR6S8lZMNYuqBNseGp4ZZfZ9.jpg


the data frame hiving missing values in genre column too since i am going to add all lists not droping missing values in genre column

In [31]:
movies[movies["genre"].str.len() == 0].shape

(231, 6)

converting overview column into list for adding all lists to crerate new columns called tags

In [32]:
movies['overview']=movies['overview'].apply(lambda x: x.split())

In [33]:
movies

,id,title,genre,overview,actors,poster_path
0,579974,RRR,"[Action, Drama]","[A, fictional, history, of, two, legendary, re...","[N. T. Rama Rao Jr., Ram Charan, Ajay Devgn]",/nEufeZlyAOLqO2brrs0yeF1lgXO.jpg
1,256040,Bāhubali: The Beginning,"[Action, Adventure, Drama, Fantasy]","[The, young, Shivudu, is, left, as, a, foundli...","[Prabhas, Rana Daggubati, Tamannaah Bhatia]",/9BAjt8nSSms62uOVYn1t3C3dVto.jpg
2,690957,Pushpa: The Rise - Part 1,"[Action, Drama, Thriller]","[Pushpa, Raj, is, a, coolie, who, rises, in, t...","[Allu Arjun, Fahadh Faasil, Rashmika Mandanna]",/r1yAzVQNbCbPTbB3GZFour9Qr0t.jpg
3,350312,Bāhubali 2: The Conclusion,"[Action, Adventure, Fantasy]","[When, Mahendra,, the, son, of, Bāhubali,, lea...","[Prabhas, Rana Daggubati, Sathyaraj]",/21sC2assImQIYCEDA84Qh9d1RsK.jpg
4,901277,CSI Sanatan,"[Crime, Thriller]","[When, a, buisness, tycoon, Vikram, Chakravart...","[Aadi Saikumar, Misha Narang, Ali Reza]",/yscCuJsNYDh2vHc6VrASnZcbaDw.jpg
...,...,...,...,...,...,...
2296,279110,Master,"[Action, Drama]","[A, popular, college, professor, is, unable, t...","[Chiranjeevi, Sakshi Shivananda, Puneet Issar]",/dZwuPQ3xU6jVpM4V2x8f2VeZYxt.jpg
2299,278035,Swati Kiranam,"[Drama, Family]","[An, egotistical, music, teacher, envious, of,...","[Mammootty, Radhika Sarathkumar, Master Manjun...",/nMtsgVTfaCdVQZOXx1Yh8Himj5M.jpg
2300,274754,Andagadu,[Comedy],"[Though, he's, successful, in, his, career, co...","[Rajendra Prasad, Damini, Chandramohan]",/iR3pxOIigseQRfqCAmr4yKcLu7f.jpg
2302,267689,Billa Ranga,[Thriller],"[Billa, Ranga, is, action, thriller, based, mo...","[Chiranjeevi, Mohan Babu, Swapna]",/4k7oR6S8lZMNYuqBNseGp4ZZfZ9.jpg


removing space between two words so that it won't consider single name or identity into two separate words

In [34]:
movies['genre']=movies['genre'].apply(lambda x:[i.replace(' ','')for i in x])
movies['overview']=movies['overview'].apply(lambda x:[i.replace(' ','')for i in x])
movies['actors']=movies['actors'].apply(lambda x:[i.replace(' ','')for i in x])


In [35]:
movies.head()

,id,title,genre,overview,actors,poster_path
0,579974,RRR,"[Action, Drama]","[A, fictional, history, of, two, legendary, re...","[N.T.RamaRaoJr., RamCharan, AjayDevgn]",/nEufeZlyAOLqO2brrs0yeF1lgXO.jpg
1,256040,Bāhubali: The Beginning,"[Action, Adventure, Drama, Fantasy]","[The, young, Shivudu, is, left, as, a, foundli...","[Prabhas, RanaDaggubati, TamannaahBhatia]",/9BAjt8nSSms62uOVYn1t3C3dVto.jpg
2,690957,Pushpa: The Rise - Part 1,"[Action, Drama, Thriller]","[Pushpa, Raj, is, a, coolie, who, rises, in, t...","[AlluArjun, FahadhFaasil, RashmikaMandanna]",/r1yAzVQNbCbPTbB3GZFour9Qr0t.jpg
3,350312,Bāhubali 2: The Conclusion,"[Action, Adventure, Fantasy]","[When, Mahendra,, the, son, of, Bāhubali,, lea...","[Prabhas, RanaDaggubati, Sathyaraj]",/21sC2assImQIYCEDA84Qh9d1RsK.jpg
4,901277,CSI Sanatan,"[Crime, Thriller]","[When, a, buisness, tycoon, Vikram, Chakravart...","[AadiSaikumar, MishaNarang, AliReza]",/yscCuJsNYDh2vHc6VrASnZcbaDw.jpg


creating new columns called tags which will include genre ,overview ,actors

In [36]:
movies['tags']=movies['genre']+movies['overview']+movies['actors']

In [37]:
new_df=movies[['id','title','tags']]

joining space with nothing in tags column

In [38]:
new_df['tags']=new_df['tags'].apply(lambda x:" ".join(x))

C:\Users\User\AppData\Local\Temp\ipykernel_13660\487797088.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags']=new_df['tags'].apply(lambda x:" ".join(x))


lowering all words in corpus

In [39]:
new_df['tags']=new_df['tags'].apply(lambda x: x.lower())

C:\Users\User\AppData\Local\Temp\ipykernel_13660\2543325826.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags']=new_df['tags'].apply(lambda x: x.lower())


creating countvectorizer object 

In [40]:
cv = CountVectorizer(max_features=5000,stop_words='english')

transforming tags columns into sparce matrix

In [41]:
vector = cv.fit_transform(new_df['tags']).toarray()

In [42]:
vector

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [43]:
cv.get_feature_names_out()

array(['10', '100', '10th', ..., 'yuvraj', 'zakeer', 'zamindar'],
      dtype=object)

some words with same meaning are repeating so applying stemming to overcame that problem

In [44]:
from nltk.stem.porter import  PorterStemmer

In [45]:
ps=PorterStemmer()

In [46]:
def stemming(text):
    a=[]
    for i in text.split():
        ps.stem(i)
        a.append(ps.stem(i))
        
    
    return ' '.join(a)

In [47]:
new_df['tags'][0]

"action drama a fictional history of two legendary revolutionaries' journey away from home before they began fighting for their country in the 1920s. n.t.ramaraojr. ramcharan ajaydevgn"

In [48]:
stemming(
"action drama a fictional history of two legendary revolutionaries' journey away from home before they began fighting for their country in the 1920s. n.t.ramaraojr. ramcharan ajaydevgn")

"action drama a fiction histori of two legendari revolutionaries' journey away from home befor they began fight for their countri in the 1920s. n.t.ramaraojr. ramcharan ajaydevgn"

applying stemming for tags column 

In [49]:
new_df['tags']=new_df['tags'].apply(stemming)

C:\Users\User\AppData\Local\Temp\ipykernel_13660\226593929.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags']=new_df['tags'].apply(stemming)


In [50]:
vector = cv.fit_transform(new_df['tags']).toarray()

In [51]:
vector

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [52]:
cv.get_feature_names_out()

array(['000', '10', '100', ..., 'yuvraj', 'zakeer', 'zamindar'],
      dtype=object)

creating cosign similarity obeject with paasing vector as input to it

In [53]:
similarity = cosine_similarity(vector)

In [54]:
similarity.shape

(1682, 1682)

In [55]:
similarity[0]

array([1.        , 0.08164966, 0.12524486, ..., 0.03580574, 0.04244764,
       0.04637389])

creating a function which will recomend 6 movies similar to the given input movie

In [56]:
sorted(list(enumerate(similarity[4])),reverse=True,key=lambda x:x[1])[1:6]

[(1422, 0.3418817293789138),
 (682, 0.2727723627949905),
 (237, 0.27003086243366087),
 (945, 0.26726124191242434),
 (946, 0.2594372608313854)]

In [57]:
def recomend(movie):
    movie_index=new_df[new_df['title']==movie].index[0]
    distance=similarity[movie_index] 
    movie_list=sorted(list(enumerate(distance)),reverse=True,key=lambda x:x[1])[1:8]
    

    for i in movie_list:
       
        print(new_df.iloc[i[0]].title)

In [58]:
recomend('Arjun Reddy')

Butta Bomma
Chirutha
Kumari 21F
Dhada
Kathi
Pelli SandaD
Ashok


pickling movies dataframe and similrity obejct for web app building 

In [59]:
pickle.dump(movies.to_dict(),open('movies with posters.pkl','wb'))

In [60]:
pickle.dump(similarity,open('similarity.pkl','wb'))